In [2]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os
import pandas as pd
import geopandas as gpd

In [48]:
import sys
sys.path.append('..')
from src.plotting import custom_cmap

from tueplots import bundles
from tueplots.constants.color import rgb
from matplotlib.colors import LinearSegmentedColormap

from pathlib import Path
# Add the parent directory to the sys.path (without this we cannot import constants or scripts)
sys.path.insert(0, str(Path.cwd().parent))

from src.utils import get_iso3_gbd

import warnings

# Suppress FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [5]:
blue = rgb.tue_blue
red = rgb.tue_red
# make red a bit lighter, but not towards pink, less blue
red = (red[0]*1.5, red[1] * 0.8, red[2] * 0.8)
# Create a colormap
cmap = LinearSegmentedColormap.from_list('blue_to_red', [blue, red])


In [29]:
# Load the world map data using geopandas
plt.rcParams.update(bundles.icml2022(column="full", ncols=2, nrows=1))

def plot_world_map(filename, data, country_col, measured_col, label, year, title, vmin, vmax):
    
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # Merge your data with the geopandas DataFrame using country names
    merged = world.merge(data[data.year==year], how='left', left_on='iso_a3', right_on=country_col)

    # Fill NaN values (countries with no deaths) with 0
    merged[measured_col] = merged[measured_col].fillna(0)
    
    # Plotting the map
    # fig, ax = plt.subplots(1, 1, figsize=(15, 10))

    
    ax = merged.plot(column=measured_col, cmap=cmap, linewidth=0.2, edgecolor='0.2', legend=True,
                legend_kwds={'label': label, 'orientation': "horizontal"}, vmin=vmin, vmax=vmax,
               )
    ax.set_title(title + ' ' + str(year))

    ax.axis('off')

    chart = ax.get_figure()
    chart.savefig(filename, dpi=600)
    plt.close()
    # return ax

In [56]:
def generate_animation(df, country_col, measured_col, label, gif_name, title):
    vmin = df[measured_col].min()
    vmax = df[measured_col].max()
    filenames = []
    for year in range(1990, 2020):
        filename = f'year_{year}.png'
        plot_world_map(filename, df, country_col=country_col, measured_col=measured_col,  label=label, year=year, title=title, vmin=vmin, vmax=vmax)
        filenames.append(filename)

    # Create gif
    with imageio.get_writer(f'output/animations/{gif_name}.gif', mode='I', duration=0.8) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove files
    for filename in filenames:
        os.remove(filename)

In [57]:
df = pd.read_csv('../data/final/gbd_cardiovascular_allAges_final.csv')
death_df = df[(df.measure_name == "Deaths")]
death_df = death_df.rename({'Country Code': 'iso_a3'}, axis=1)
death_df.head()

,measure_name,location_name,metric_name,year,val,iso_a3
0,Deaths,Global,Percent,1990,0.258754,GLB
1,Deaths,Global,Rate,1990,225.584376,GLB
2,Deaths,Timor-Leste,Percent,1990,0.080068,TLS
3,Deaths,Timor-Leste,Rate,1990,79.809246,TLS
4,Deaths,Japan,Percent,1990,0.348712,JPN


In [58]:
generate_animation(death_df, country_col='iso_a3', measured_col='val', label='Death Rate', gif_name='cvd_deaths', title='Death Rate due to Cardiovascular Diseases')

C:\Users\HP\AppData\Local\Temp\ipykernel_12716\2822330716.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [59]:
df = pd.read_csv('../data/final/gbd_cardiovascular_allAges_final.csv')
death_df = df[(df.measure_name == "Incidence")]
death_df = death_df.rename({'Country Code': 'iso_a3'}, axis=1)
generate_animation(death_df, country_col='iso_a3', measured_col='val', label='Incidence Rate', gif_name='cvd_incidence', title='Incidence Rate of Cardiovascular Diseases')

C:\Users\HP\AppData\Local\Temp\ipykernel_12716\2822330716.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [60]:
df = pd.read_csv('../data/final/gbd_IschemicHeartDisease_DeathsIncidence.csv')
df['iso_a3'] = df['location_name'].apply(get_iso3_gbd)
df = df[df['measure_name'] == 'Deaths']
generate_animation(df, country_col='iso_a3', measured_col='Value', label='Death Rate', gif_name='ihd_deaths', title='Death Rate due to Ischemic Heart Disease')

C:\Users\HP\AppData\Local\Temp\ipykernel_12716\2822330716.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [61]:
df = pd.read_csv('../data/final/gbd_IschemicHeartDisease_DeathsIncidence.csv')
df['iso_a3'] = df['location_name'].apply(get_iso3_gbd)
df = df[df['measure_name'] == 'Incidence']
generate_animation(df, country_col='iso_a3', measured_col='Value', label='Death Rate', gif_name='ihd_incidence', title='Incidence Rate of Ischemic Heart Disease')

C:\Users\HP\AppData\Local\Temp\ipykernel_12716\2822330716.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [62]:
df = pd.read_csv('../data/final/gbd_ihd_stroke.csv')
df['iso_a3'] = df['location_name'].apply(get_iso3_gbd)
df = df[df['measure_name'] == 'Deaths']
df = df[df['cause_name'] == 'Stroke']
generate_animation(df, country_col='iso_a3', measured_col='val', label='Death Rate', gif_name='stroke_deaths', title='Death Rate due to Stroke')

C:\Users\HP\AppData\Local\Temp\ipykernel_12716\2822330716.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [63]:
df = pd.read_csv('../data/final/gbd_ihd_stroke.csv')
df['iso_a3'] = df['location_name'].apply(get_iso3_gbd)
df = df[df['measure_name'] == 'Incidence']
df = df[df['cause_name'] == 'Stroke']
generate_animation(df, country_col='iso_a3', measured_col='val', label='Incidence Rate', gif_name='stroke_incidence', title='Incidence rate of Stroke')

C:\Users\HP\AppData\Local\Temp\ipykernel_12716\2822330716.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
